In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANG_SMITH")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229")

In [3]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# 크롤링
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# 지식 베이스 생성 및 벡터 저장
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [5]:
# 프롬프트 템플릿과 rag chain 생성

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


##--- Lagacy ----##

In [6]:
# history기반 대화 RAG 리트리버 만들기

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [7]:
# 일반적인 RAG 체인 만들기. 단, chat hisory가 프롬프트화 됨

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [8]:
# 히스토리 기반 대화

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)
second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

There are several common approaches to task decomposition:

User Input: What are common ways of doing task decomposition?

Task Planning:
1. Identify different techniques for task decomposition
2. Explain each technique briefly with examples
3. Highlight the pros and cons of different approaches

Model Selection: 
1. Leverage my own knowledge about project management and task planning
2. Refer to the given context about chain of thought prompting for examples

Task Execution:
Some common ways of performing task decomposition include:

1. Top-down approach: This involves starting with the overall big task and successively breaking it down into smaller sub-tasks or components in a hierarchical manner. For example, when planning a wedding, the top-level task is "Plan the wedding", which can be broken down into venues, catering, guest list, etc.

2. Bottom-up approach: Here you start by identifying the basic actionable steps first, and then combine related steps into higher-level tasks or 

In [9]:
# 히스토리 관리

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [10]:
# 첫번째 히스토리 생성

conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps. It involves identifying the different components or subgoals required to achieve the overall objective.\n\nTask Planning: The key steps involved in task decomposition are:\n1. Understand the overall task or goal\n2. Identify the main subtasks or components needed to accomplish the goal\n3. Break down each subtask further into smaller steps if needed\n4. Determine the order or sequence in which the subtasks should be performed\n\nModel Selection: To perform task decomposition for the given query, I will use my own knowledge and reasoning capabilities as a large language model.\n\nTask Execution: The retrieved context explains task decomposition techniques used by AI systems like Chain of Thought (CoT) and Tree of Thoughts. CoT prompts the model to "think step-by-step" to decompose a hard task into simpler steps. Tree of Thoughts extends this by exploring multiple reasoning

In [11]:
# 히스토리 생성 제대로 되는지 확인

conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'There are a few common approaches to task decomposition:\n\n1. Top-down decomposition: Start with the overall task or goal, and break it down into major subtasks or components. Then further decompose each subtask into smaller steps until you reach an actionable level.\n\n2. Bottom-up decomposition: Identify the basic actionable steps first, and then group and sequence them into higher-level subtasks that ultimately accomplish the overall goal.\n\n3. Hierarchical decomposition: Combine aspects of top-down and bottom-up approaches. Decompose the task into major components top-down, then decompose each component bottom-up into actionable steps.\n\n4. Technique-based decomposition: Use domain knowledge, templates or specific techniques for the type of task. For example, writing an outline when decomposing a writing task.\n\n5. Question-based decomposition: Ask questions like "What are the steps to achieve X?" or "What subgoals do I need to complete X?" to prompt decomposition.\n\n6. Tool-

In [12]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps. It involves identifying the different components or subgoals required to achieve the overall objective.

Task Planning: The key steps involved in task decomposition are:
1. Understand the overall task or goal
2. Identify the main subtasks or components needed to accomplish the goal
3. Break down each subtask further into smaller steps if needed
4. Determine the order or sequence in which the subtasks should be performed

Model Selection: To perform task decomposition for the given query, I will use my own knowledge and reasoning capabilities as a large language model.

Task Execution: The retrieved context explains task decomposition techniques used by AI systems like Chain of Thought (CoT) and Tree of Thoughts. CoT prompts the model to "think step-by-step" to decompose a hard task into simpler steps. Tree of Thoughts extends this by 

In [13]:
# Agent 결합하기 위해 tool 만들기

from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post."
)
tools = [tool]

In [14]:
tool.invoke("task decomposition")

'Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.\n\nFig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The mode

In [15]:
# Agent 만들기

from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [17]:
# Tool의 사용을 유도하여 질문하기

query = "What is Task Decomposition? find document."

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("-----")

{'agent': {'messages': [AIMessage(content=[{'id': 'toolu_018V7hdT6thtyskL5H347G7H', 'input': {'query': 'Task Decomposition'}, 'name': 'blog_post_retriever', 'type': 'tool_use'}], response_metadata={'id': 'msg_014pHCLdR8CBByW5SPmPA1G5', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 253, 'output_tokens': 59}}, id='run-10db4db5-1b56-47ba-8da5-94faa63b74b5-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'Task Decomposition'}, 'id': 'toolu_018V7hdT6thtyskL5H347G7H', 'type': 'tool_call'}], usage_metadata={'input_tokens': 253, 'output_tokens': 59, 'total_tokens': 312})]}}
-----
{'tools': {'messages': [ToolMessage(content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique f

In [18]:
# langgraph로 history 저장소 만들어서 대화하기

from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [19]:
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob, nice to meet you! I'm an AI assistant created by Anthropic to be helpful, harmless, and honest. How can I assist you today?", response_metadata={'id': 'msg_01CwkqjHzzAvTsLifMR8zwp6', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 248, 'output_tokens': 38}}, id='run-4c893e0d-2848-4e51-969c-d7bae5f72531-0', usage_metadata={'input_tokens': 248, 'output_tokens': 38, 'total_tokens': 286})]}}
----


In [21]:
query = "What is Task Decomposition? find document"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'text': 'Okay, let me search the Autonomous Agents blog post for relevant information on task decomposition.', 'type': 'text'}, {'id': 'toolu_01TTvt7axuX4DvtxouzpZj9p', 'input': {'query': 'task decomposition'}, 'name': 'blog_post_retriever', 'type': 'tool_use'}], response_metadata={'id': 'msg_01Y4uBNP9yXvoyVLFpJmmrwj', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 571, 'output_tokens': 78}}, id='run-1fb5b7d2-b9d4-4d6a-81ae-c2d3fd0e420f-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'task decomposition'}, 'id': 'toolu_01TTvt7axuX4DvtxouzpZj9p', 'type': 'tool_call'}], usage_metadata={'input_tokens': 571, 'output_tokens': 78, 'total_tokens': 649})]}}
----
{'tools': {'messages': [ToolMessage(content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought ste